In [3]:
import treelite
from treelite.model_builder import (
  Metadata,
  ModelBuilder,
  PostProcessorFunc,
  TreeAnnotation,
)
import sklearn
import joblib
import tl2cgen
import numpy as np
import time

In [4]:
sklearn_path = '../model/nyc-taxi-green-dec-2016_d10_l859_n1717_20241015054511_rf.joblib'
sklearn_model = joblib.load(sklearn_path)

In [2]:
sklearn_path = '../model/nyc-taxi-green-dec-2016_d10_l859_n1717_20241015054511_out2.joblib'
sklearn_model = joblib.load(sklearn_path)

NameError: name 'joblib' is not defined

In [5]:
model = treelite.sklearn.import_model(sklearn_model)

In [6]:
toolchain = "clang"
# toolchain = "gcc"

In [7]:
tl2cgen.export_lib(model, toolchain=toolchain, libpath="../model/lib/nyc-taxi-green-dec-2016_d10_l859_n1717_20241015054511_rf.so")
predictor = tl2cgen.Predictor("../model/lib/nyc-taxi-green-dec-2016_d10_l859_n1717_20241015054511_rf.so")

[15:59:48] /home/conda/feedstock_root/build_artifacts/tl2cgen_1709780361179/work/src/compiler/ast/split.cc:30: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.


In [8]:
npy_path = "../data/nyc-taxi-green-dec-2016_1G.npy"
data = np.load(npy_path)
dmat = tl2cgen.DMatrix(data=data, dtype='float32', missing=None)


In [10]:
tl2cgen.annotate_branch(model, dmat, '../model/ccode/branch.json')

In [13]:
tl2cgen.generate_c_code(model, dirpath="../model/ccode/",params={})

[16:03:40] /home/conda/feedstock_root/build_artifacts/tl2cgen_1709780361179/work/src/compiler/ast/split.cc:30: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.


In [30]:
start = time.perf_counter()
predictor.predict(dmat)
end = time.perf_counter()
print(end-start)

0.22199486001045443
